In [1]:
%load_ext autoreload
%autoreload 2

# Getting the pockets of a protein:



The libraries `openpocket`, `molsysmt` and `pyunitwizard` are imported:

In [2]:
import molsysmt as msm       # To work with molecular systems
import pyunitwizard as puw   # To work with quantities
import openpocket as opoc    # To get the cavities, pockets, grooves and channels of a molecule

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

Let's get the coordinates of the protein with PDB id: 2LAO

In [4]:
molsys = msm.convert('2LAO', selection='molecule_type=="protein"')

/home/diego/MyMachine/locals/miniconda3/envs/OpenPocket@dprada_3.9/lib/python3.9/site-packages/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:32: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)
/home/diego/MyMachine/locals/miniconda3/envs/OpenPocket@dprada_3.9/lib/python3.9/site-packages/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:32: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)


The protein can be shown in this jupyter notebook with the assitance of `molsysmt` and `nglview`:

In [5]:
msm.view(molsys)

/home/diego/MyMachine/locals/miniconda3/envs/OpenPocket@dprada_3.9/lib/python3.9/site-packages/nglview/adaptor.py:623: NotDigestedArgumentWarning: The digest from molsysmt.basic.convert.convert argument was not digested.Check  for more information. If you still need help, open a new issue in https://github.com/uibcdf/MolSysMT/issues.
  self.pdb = msm.convert(molsys, to_form='string:pdb_text', selection=selection, structure_indices=0, digest=False)
/home/diego/MyMachine/locals/miniconda3/envs/OpenPocket@dprada_3.9/lib/python3.9/site-packages/nglview/adaptor.py:624: NotDigestedArgumentWarning: The digest from molsysmt.basic.get.get argument was not digested.Check  for more information. If you still need help, open a new issue in https://github.com/uibcdf/MolSysMT/issues.
  _coordinates = msm.get(molsys, element='system', structure_indices=structure_indices, coordinates=True, digest=False)


NGLWidget()

Let's get the coordinates of all atoms in the protein:

In [6]:
coordinates = msm.get(molsys, coordinates=True)

In [7]:
coordinates.shape

(1, 1822, 3)

In [8]:
type(coordinates)

pint.util.Quantity

In [9]:
coordinates

Magnitude,[[[5.143599999999999 4.427 1.0041] [5.0954999999999995 4.4765999999999995 0.8768] [4.9536999999999995 4.5148 0.9132999999999999] ... [3.9659999999999993 7.1579999999999995 2.8547999999999996] [4.0333 7.073399999999999 3.042] [4.2606 6.887499999999999 2.5469999999999997]]]
Units,nanometer


As you can see, the object `coordinates` y is a quantity with shape [n_structures, n_atoms, 3]. OpenPocket is not ready to work with quantities, and the `AlphaSpheres` class requires an ndarray with shape [n_atoms, 3]. Let's then get the value of the coordinates with the proper shape:

In [11]:
coordinates = puw.get_value(coordinates[0])

In [12]:
coordinates.shape

(1822, 3)

The set of alpha spheres defined by the 3 dimensional coordinates of the 1822 atoms in the system can already be obtained:

In [13]:
alphaspheres = opoc.alpha_spheres.AlphaSpheres(coordinates)

In [14]:
alphaspheres.n_alpha_spheres

11834

## How many isolated alpha-spheres we have?

In [ ]:
from scipy.spatial.distance import euclidean

In [ ]:
edges=[]

for ii in tqdm(range(alphaspheres.n_alpha_spheres)):
    for jj in range(ii+1, alphaspheres.n_alpha_spheres):
        distance = euclidean(alphaspheres.centers[ii], alphaspheres.centers[jj])
        if distance < alphaspheres.radii[ii]+alphaspheres.radii[jj]:
            edges.append([ii,jj])

In [ ]:
len(edges)

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph(edges)

In [ ]:
G.number_of_edges()

In [ ]:
G.number_of_nodes()

In [ ]:
connected = list(nx.connected_components(G))

In [ ]:
len(connected)

And if the the small alpha-spheres are removed?

In [17]:
alphaspheres.remove_small_alpha_spheres(0.2)

In [18]:
alphaspheres.n_alpha_spheres

8905

What are we going to do with them?
(hint: we should remove the small and big alpha spheres, and the resulting ones have to be clusterized)